We discussed Document Loading and Splitting as well as Storage and Retrieval.

We then showed how Retrieval can be used for output generation in Q+A using RetrievalQA chain.

In [1]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

If you wish to experiment on the `LangSmith platform` (previously known as LangChain Plus):

 * Go to [LangSmith](https://www.langchain.com/langsmith) and sign up
 * Create an api key from your account's settings
 * Use this api key in the code below 

In [2]:
persist_directory = 'D:\Github\Chat-with-your-docs\Vectorstores-and-Embeddings\docs\chroma'
embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

#print resultados
print(vectordb._collection.count())

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\vinic\AppData\Local\Temp\ipykernel_1476\2951728076.py:1: SyntaxWarning: invalid escape sequence '\G'
  persist_directory = 'D:\Github\Chat-with-your-docs\Vectorstores-and-Embeddings\docs\chroma'
C:\Users\vinic\AppData\Local\Temp\ipykernel_1476\2951728076.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
d:\Github\Chat-with-your-docs\conda-gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https

209


In [3]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [4]:
# Testando a predição do modelo
llm = Ollama(model='gemma2:2b', temperature=0)
llm.predict("Hello world!")

C:\Users\vinic\AppData\Local\Temp\ipykernel_1476\2655735778.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='gemma2:2b', temperature=0)
C:\Users\vinic\AppData\Local\Temp\ipykernel_1476\2655735778.py:3: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Hello world!")


'Hello to you too! 👋  \n\nWhat can I do for you today? 😊 \n'

# Build prompt

In [5]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "Is probability a class topic?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

C:\Users\vinic\AppData\Local\Temp\ipykernel_1476\3308332923.py:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


'Yes, the text states that the professor assumes familiarity with basic probability and statistics.  \n\nthanks for asking! \n'

# Memory

In [6]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\vinic\AppData\Local\Temp\ipykernel_1476\1956280058.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


# ConversationalRetrievalChain

In [7]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [8]:
question = "Is probability a class topic?"
result = qa({"question": question})

In [9]:
result['answer']

'Yes, the text states "I also assume familiarity with basic probability and statistics."  So yes, probability is a class topic. \n'

In [10]:
question = "why are those prerequesites needed?"
result = qa({"question": question})

In [11]:
result['answer']

"The prerequisites listed (basic probability and statistics, linear algebra) are necessary to understand the concepts of probability and their application in machine learning.  Here's why:\n\n* **Probability:** The course will be using probability theory to model data and make predictions. Understanding basic probability concepts like random variables, expectation, variance, and conditional probability is crucial for grasping these models.\n* **Statistics:** Probability is closely intertwined with statistics.  The course assumes familiarity with statistical concepts like sampling distributions, hypothesis testing, and confidence intervals, which are essential for analyzing data and drawing conclusions from it. \n* **Linear Algebra:** Linear algebra provides the mathematical framework for working with matrices and vectors, which are fundamental in machine learning algorithms. Understanding matrix operations, vector spaces, eigenvalues, and eigenvectors is necessary to understand how the